# ATTACKS

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['SCIPY_USE_PROPACK'] = "True"
 
threads = "64"
os.environ["OMP_NUM_THREADS"] = threads
os.environ["OPENBLAS_NUM_THREADS"] = threads
os.environ["MKL_NUM_THREADS"] = threads
os.environ["VECLIB_MAXIMUM_THREADS"] = threads
os.environ["NUMEXPR_NUM_THREADS"] = threads

In [2]:
#import torchattacks

import torch
from tensordict import TensorDict
from tensordict import MemoryMappedTensor as MMT

import abc  

/srv/newpenny/conda/envs/faiss-LC/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
# python stuff

from pathlib import Path as Path
from numpy.random import randint
from tqdm import tqdm

# Our stuff
from datasets.cifar import Cifar
from models.model_wrap import ModelWrap
# from adv_atk.attacks_base import fds, ftd
from coreVectors.coreVectors import CoreVectors 
from coreVectors.svd_coreVectors import reduct_matrices_from_svds as parser_fn

# from credibility import get_credibility

# torch stuff
from torchvision.models import vgg16, VGG16_Weights

use_cuda = torch.cuda.is_available()
device = torch.device(auto_cuda('utilization')) if use_cuda else torch.device("cpu")
print(f"Using {device} device")

# use_cuda = torch.cuda.is_available()
# cuda_index = 1
# device = torch.device(f"cuda:{cuda_index}" if use_cuda else "cpu")
# print(f"Using {device} device")

#--------------------------------
# Dataset 
#--------------------------------
# model parameters
name_model = 'vgg16'
dataset = 'CIFAR100' 
seed = 29
bs = 64
data_path = '/srv/newpenny/dataset/CIFAR100'

ds = Cifar(dataset=dataset, data_path=data_path)
ds.load_data(
        batch_size = bs,
        data_kwargs = {'num_workers': 4, 'pin_memory': True},
        seed = seed,
        )

Using cpu device
dataset: CIFAR100
Files already downloaded and verified
Files already downloaded and verified


{'train': <torch.utils.data.dataloader.DataLoader at 0x7fe46ffb7890>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7fe46fe136d0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7fe4737b2a50>}

In [ ]:
#--------------------------------
# Model 
#--------------------------------
pretrained = True
model_dir = '/srv/newpenny/XAI/models'
# model_name = f'{name_model}_pretrained={pretrained}_dataset={dataset}-'\
# f'augmented_policy=CIFAR10_bs={bs}_seed={seed}.pth'

model_name = 'LM_model=vgg16_dataset=CIFAR100_augment=True_optim=SGD_scheduler=LROnPlateau.pth'

nn = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
in_features = 4096
num_classes = len(ds.get_classes()) 
nn.classifier[-1] = torch.nn.Linear(in_features, num_classes)
model = ModelWrap(device=device)
model.set_model(model=nn, path=model_dir, name=model_name, verbose=True)

target_layers = [
            #'classifier.0',
            # 'classifier.3',
            'features.14'
            ]
model.set_target_layers(target_layers=target_layers, verbose=True)

model._model

# direction = {'save_input':True, 'save_output':False}
# model.add_hooks(**direction, verbose=False) 

# dry_img, _ = ds._train_ds.dataset[0]
# dry_img = dry_img.reshape((1,)+dry_img.shape)
# model.dry_run(x=dry_img)

# #--------------------------------
# # CoreVectors 
# #--------------------------------
# loaders = ds.get_dataset_loaders()

# cvs_name = 'corevectors'
# cvs_path = Path.cwd()/f'../data/_corevectors_l/{dataset}'

# corevecs = CoreVectors(
#         path = cvs_path,
#         name = cvs_name,
#         model = model,
#         device = device
#         )

# # copy dataset to coreVect dataset
# with corevecs as cv:
#     cv.get_coreVec_dataset(
#             loaders = loaders,
#             verbose = True
#             ) 
#     cv.get_activations(
#                 loaders = loaders,
#                 verbose = True
#                 )


/home/lorenzocapelli/repos/XAI/src/models/model_wrap.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._checkpoint = torch.load(file, map_location=self.device)



-----------------
checkpoint
-----------------
state_dict keys: 
 odict_keys(['features.0.weight', 'features.0.bias', 'features.2.weight', 'features.2.bias', 'features.5.weight', 'features.5.bias', 'features.7.weight', 'features.7.bias', 'features.10.weight', 'features.10.bias', 'features.12.weight', 'features.12.bias', 'features.14.weight', 'features.14.bias', 'features.17.weight', 'features.17.bias', 'features.19.weight', 'features.19.bias', 'features.21.weight', 'features.21.bias', 'features.24.weight', 'features.24.bias', 'features.26.weight', 'features.26.bias', 'features.28.weight', 'features.28.bias', 'classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.6.weight', 'classifier.6.bias']) 

epoch 58
initial_lr 0.001
final_lr 1e-05
train_accuracy 93.895
val_accuracy 95.53
loss_plot b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\xe0\x08\x06\x00\x00\x005\xd1\xdc\xe4\x00\x00\x009tEXtSoftware\x00Matplotlib version3.9.2, 

## PGD

In [15]:
from adv_atk.PGD import myPGD

In [16]:
Path.cwd()

PosixPath('/home/lorenzocapelli/repos/XAI/src')

In [24]:
name = 'PGD'
path = f'/srv/newpenny/XAI/generated_data/attacks/{name}'

In [25]:
kwargs = {'model': model._model,
          'eps' : 8/255, 
          'alpha' : 2/255, 
          'steps' : 10,
          'device' : device,
          'path' : path,
          'name' : name,
          'dl' : loaders,
          'name_model' : name_model,
          'verbose' : True,
          'mode' : 'random', 
}

In [26]:
atk = myPGD(**kwargs)

---------- Attack PGD init

File /srv/newpenny/XAI/generated_data/attacks/PGD/model_vgg16/eps_0.03/alpha_0.01/steps_10/random_start_True exists.


In [27]:
atk.get_ds_attack()


 ---- Getting data from train



  0%|                                                   | 0/625 [00:00<?, ?it/s]


AttributeError: 'myPGD' object has no attribute 'atk'

## BIM

In [15]:
from adv_atk.BIM import myBIM

In [16]:
name = 'BIM'
path = f'srv/newpenny/XAI/generated_data/attacks/{name}'

In [22]:
kwargs = {'model': model._model,
          'eps' : 8/255, 
          'alpha' : 2/255, 
          'steps' : 10,
          'device' : device,
          'path' : path,
          'name' : name,
          'dl' : loaders,
          'name_model' : name_model,
          'verbose' : True,
          'mode' : 'random'
}

In [24]:
atk = myBIM(**kwargs)

---------- Attack BIM init

File srv/newpenny/XAI/generated_data/attacks/BIM/model_vgg16/eps_0.03/alpha_0.01/steps_10 exists.


FileNotFoundError: [Errno 2] No such file or directory: 'srv/newpenny/XAI/generated_data/attacks/BIM/model_vgg16/eps_0.03/alpha_0.01/steps_10/train/meta.json'

In [19]:
atk._atkds

AttributeError: 'myBIM' object has no attribute '_atkds'

In [14]:
atk.get_ds_attack()


 ---- Getting data from train

loader n_samples:  40000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [11:53<00:00,  1.14s/it]



 ---- Getting data from val

loader n_samples:  10000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [02:49<00:00,  1.08s/it]



 ---- Getting data from test

loader n_samples:  10000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [02:48<00:00,  1.07s/it]


## CW

In [30]:
from adv_atk.CW import myCW

In [31]:
name = 'CW'
path = f'srv/newpenny/XAI/generated_data/attacks/{name}'

In [36]:
path

'srv/newpenny/XAI/generated_data/attacks/CW'

In [32]:
loader = {}
loader['test'] = loaders['test']

In [33]:
kwargs = {'model': model._model,
          'device' : device,
          'path' : path,
          'name' : name,
          'dl' : loader,
          'name_model' : name_model,
          'verbose' : True,
          'nb_classes' : ds.config['num_classes'],
          'confidence': 0,
          'c_range': (1e-3, 1e10),
          'max_steps': 1000,
          'optimizer_lr': 1e-2,
          'verbose': True,}

In [34]:
atk = myCW(**kwargs)

---------- Attack CW init



In [11]:
atk.get_ds_attack()


 ---- Getting data from test



100%|██████████████████████████████████████| 157/157 [4:42:15<00:00, 107.87s/it]


Saving test to /home/lorenzocapelli/repos/XAI/src/../data/attacks/CW/model_vgg16/confidence_0/c_range_(0.001, 10000000000.0)/max_steps_1000/optimizer_lr_0.01/test.


## DeepFool

In [6]:
from adv_atk.DeepFool import myDeepFool

In [7]:
name = 'DeepFool'
path = Path.cwd()/f'../data/attacks/{name}'

In [8]:
loader = {}
loader['test'] = loaders['test']

In [9]:
kwargs = {'model': model._model,
          'steps' : 50,
          'overshoot' : 0.02,
          'device' : device,
          'path' : path,
          'name' : name,
          'dl' : loader,
          'name_model' : name_model,
          'verbose' : True,
          }

In [10]:
atk = myDeepFool(**kwargs)

---------- Attack DeepFool init

{'test': <torch.utils.data.dataloader.DataLoader object at 0x7f44d7a7eb50>}


In [11]:
atk.get_ds_attack()


 ---- Getting data from test

10000
loader n_samples:  10000


100%|██████████████████████████████████████| 157/157 [4:46:18<00:00, 109.42s/it]


Saving test to /home/lorenzocapelli/repos/XAI/src/../data/attacks/DeepFool/model_vgg16/steps_50/overshoot_0.02/test.


# Computing corevectors for the attacks

In [28]:
from torch.utils.data import DataLoader

from pathlib import Path as Path
from numpy.random import randint
from tqdm import tqdm

# Our stuff
from datasets.cifar import Cifar
from models.model_wrap import ModelWrap
from coreVectors.coreVectors import CoreVectors 
from coreVectors.svd_coreVectors import reduct_matrices_from_svds as parser_fn

In [29]:
atk_loaders = {key: DataLoader(value, batch_size=bs, collate_fn = lambda x: x, shuffle=False) for key, value in atk._atkds.items()}

In [34]:
cvs_name = 'corevectors'
cvs_path = Path.cwd()/f'../data/corevectors_{name}'

corevecs = CoreVectors(
        path = cvs_path,
        name = cvs_name,
        model = model,
        device = device
        )

# copy dataset to coreVect dataset
with corevecs as cv:
    cv.get_coreVec_dataset(
            loaders = atk_loaders,
            verbose = True,
            parser = ftd,
            key_list = list(atk._atkds['train'].keys())
            ) 
    cv.get_activations(
                loaders = atk_loaders,
                verbose = True
                )


 ---- Getting data from train

/home/lorenzocapelli/repos/XAI/src/../data/corevectors_attack/corevectors.train
File /home/lorenzocapelli/repos/XAI/src/../data/corevectors_attack/corevectors.train exists. Loading from disk.
loaded n_samples:  40000

 ---- Getting data from val

/home/lorenzocapelli/repos/XAI/src/../data/corevectors_attack/corevectors.val
File /home/lorenzocapelli/repos/XAI/src/../data/corevectors_attack/corevectors.val exists. Loading from disk.
loaded n_samples:  10000

 ---- Getting data from test

/home/lorenzocapelli/repos/XAI/src/../data/corevectors_attack/corevectors.test
File /home/lorenzocapelli/repos/XAI/src/../data/corevectors_attack/corevectors.test exists. Loading from disk.
loaded n_samples:  10000

 ---- Getting activations for train

adding in act tensorDict
Out activations exist.
allocating in act layer:  classifier.0
bss:  torch.Size([40000]) torch.Size([40000])
Layers to save:  ['classifier.0']
Computing activations


100%|███████████████████████████████████████| 625/625 [2:11:35<00:00, 12.63s/it]



 ---- Getting activations for val

adding in act tensorDict
Out activations exist.
allocating in act layer:  classifier.0
bss:  torch.Size([10000]) torch.Size([10000])
Layers to save:  ['classifier.0']
Computing activations


100%|█████████████████████████████████████████| 157/157 [14:05<00:00,  5.39s/it]



 ---- Getting activations for test

adding in act tensorDict
Out activations exist.
allocating in act layer:  classifier.0
bss:  torch.Size([10000]) torch.Size([10000])
Layers to save:  ['classifier.0']
Computing activations


100%|█████████████████████████████████████████| 157/157 [14:02<00:00,  5.36s/it]

closing train
closing val
closing test
